<div class="alert alert-block alert-info">
    <p><b>Курс: </b>Big Data for Data Science</p>
    <p><b>Раздел: </b>5. Spark ML - задачи</p>
</div>

In [15]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.regression import LinearRegression

In [3]:
import random
import numpy as np
#set seed
random.seed(1234)
np.random.seed(1234)

In [4]:
spark = SparkSession.builder.appName("PySparkML").getOrCreate()

In [5]:
spark

## №1 Линейная регрессия

Загрузите данные для применения линейной регрессии <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/linear_regression.parquet">linear_regression.parquet</a>

In [7]:
lr_df = spark.read.parquet('_data\linear_regression.parquet')

In [8]:
lr_df.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



Создайте учителя линейной регресии со следующими параметрами:
maxIter=20
regParam=0.5
elasticNetParam=0.75<br>
<a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression">LinearRegression</a>

In [9]:
lr = LinearRegression(maxIter=20, regParam=0.5, elasticNetParam=0.75)

Выполните обучения на загруженных данных и сохраните результат в переменную.

In [10]:
lrModel = lr.fit(lr_df)

Найдите следующие параметры полученной модели rootMeanSquaredError (RMSE), r2 и округлити их до 3его знака.

In [23]:
lr_prediction = lrModel.transform(lr_df)
evaluator_RMSE = RegressionEvaluator(metricName='rmse')
evaluator_r2 = RegressionEvaluator(metricName='r2')

print(f'RMSE = {round(evaluator_RMSE.evaluate(lr_prediction), 3)}, r2 = {round(evaluator_r2.evaluate(lr_prediction), 3)}')

RMSE = 10.217, r2 = 0.017


## №2 Кластеризация (K-Means)

Загрузите данные для применения из <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/wine.parquet">wine.parquet</a>

In [25]:
wine_df = spark.read.parquet('_data\wine.parquet')

In [26]:
wine_df.show(5)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|Customer_Segment|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+
|  14.23|      1.71|2.43|        15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92|   1065|               1|
|   13.2|      1.78|2.14|        11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4|   1050|               1|
|  13.16|      2.36|2.67|        18.6|      101|          2.8|      3.24|                 0.3|           2.81|           5.68|1.03| 3.17|   1185|               1|
|  14.37|      1.95| 2

Примените <a href="https://spark.apache.org/docs/latest/ml-features.html#vectorassembler">VectorAssembler</a> для создания вектора фич (задействуйте все свойства, кроме Customer_Segment).

In [27]:
feature = VectorAssembler(inputCols=wine_df.columns[0:-1], outputCol='features').transform(wine_df)

In [28]:
feature.show(5)

+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+--------------------+
|Alcohol|Malic_Acid| Ash|Ash_Alcanity|Magnesium|Total_Phenols|Flavanoids|Nonflavanoid_Phenols|Proanthocyanins|Color_Intensity| Hue|OD280|Proline|Customer_Segment|            features|
+-------+----------+----+------------+---------+-------------+----------+--------------------+---------------+---------------+----+-----+-------+----------------+--------------------+
|  14.23|      1.71|2.43|        15.6|      127|          2.8|      3.06|                0.28|           2.29|           5.64|1.04| 3.92|   1065|               1|[14.23,1.71,2.43,...|
|   13.2|      1.78|2.14|        11.2|      100|         2.65|      2.76|                0.26|           1.28|           4.38|1.05|  3.4|   1050|               1|[13.2,1.78,2.14,1...|
|  13.16|      2.36|2.67|        18.6|      101|          2.8|      3.24|       

Cоздайте учителя KMeans со следующими параметрами K=3 Seed=1<br>
Обучите модель и примените ее к тому же вектору.
Документация по <a href="https://spark.apache.org/docs/latest/ml-clustering.html#k-means">KMeans</a>

In [29]:
kmeans = KMeans().setK(3).setSeed(1)
model = kmeans.fit(feature)

Найдите силуэт с евклидовым расстоянием в квадрате для данных по вину(округлите до четвертого знака).
<br><a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.ClusteringEvaluator">ClusteringEvaluator</a>

In [33]:
wine_prediction = model.transform(feature)
evaluator = ClusteringEvaluator()
round(evaluator.evaluate(wine_prediction), 4)

0.7323

## №3 DecisionTreeClassifier

Загрузити датасет из файла <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/iris.parquet">iris.parquet</a>

In [ ]:
iris_df = #Ваш код

In [ ]:
iris_df.show(5)

Составьте из наших признаков вектор применив <a href="https://spark.apache.org/docs/latest/ml-features.html#vectorassembler">VectorAssembler</a> как колонку features.
<br> Задействуйте все признаки, кроме species, так как он является целевым.

In [ ]:
#Ваш код

Используйте <a href="https://spark.apache.org/docs/latest/ml-features.html#stringindexer">StringIndexer</a> и сделайте новый признак с именем type из целевого признака species который является категориальным.

In [ ]:
#Ваш код

In [ ]:
iris_df.show(5)

Сформируем выборки на обучение и тест.

In [ ]:
(training_data, test_data) = iris_df.randomSplit([0.8, 0.2],seed = 42)

Создайте и обучите <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#decision-tree-classifier">DecisionTreeClassifier</a> на датасете для обучения.
Полученную модель используйте над тестовым датасетом.

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
#Ваш код

Используйте <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator">MulticlassClassificationEvaluator</a> (помните что целевая фича это - type) для оценки качества модели по метрике accuracy.<br>Какая точность полученной модели?

In [ ]:
#Ваш код

## №4 Random forest

Создайте и обучите <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier">RandomForestClassifier</a> из 10 деревьев на датасете для обучения.
Полученную модель примените к тестовому датасету.


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
#Ваш код

Используйте <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator">MulticlassClassificationEvaluator</a> (помните что целевая фича это - type) для оценки качества модели по метрике accuracy.<br>Какая точность полученной модели?

In [ ]:
#Ваш код

## №5 Hyperparameter tuning

Займемся оптимизацией гиперпараметров для модели.
Примените <a href='https://spark.apache.org/docs/latest/ml-tuning.html#train-validation-split'>TrainValidationSplit</a> для оптимизации гиперпараметров используя подготовленный вами выше датасет <a href="https://github.com/AlexKbit/stepik-ds-course/raw/master/Week5/SparkML/spark-tasks/iris.parquet">iris.parquet</a> на модели <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#random-forest-classifier">RandomForestClassifier</a> совместно с <a href="https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator">MulticlassClassificationEvaluator</a>.
<br>Ваша цель определить оптимальные значения параметров из следующих диапазонов:
<br>impurity = ["entropy", "gini"]
<br>maxDepth = [2, 3, 4, 5]
<br>numTrees = [3, 6, 9, 12, 15, 18, 21]

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

In [ ]:
model = #Ваш код

In [ ]:
print('Num Trees: {}'.format(model.bestModel._java_obj.getNumTrees()))
print('Max Depth: {}'.format(model.bestModel._java_obj.getMaxDepth()))
print('Impurity: {}'.format(model.bestModel._java_obj.getImpurity()))